In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import datetime as dt
from scipy import stats
import warnings
warnings.filterwarnings("ignore")
 
%matplotlib inline

In [9]:
import os
os.path.exists("data/ccf_offline_stage1_test_revised.csv")

True

In [10]:
off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',keep_default_na=True)
off_train.columns=['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',keep_default_na=True)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
on_train = pd.read_csv('data/ccf_online_stage1_train.csv',keep_default_na=True)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']
off_train[['user_id','merchant_id','coupon_id']]=off_train[['user_id','merchant_id','coupon_id']].astype(str)
off_test[['user_id','merchant_id','coupon_id']]=off_test[['user_id','merchant_id','coupon_id']].astype(str)
on_train[['user_id','merchant_id','coupon_id']]=on_train[['user_id','merchant_id','coupon_id']].astype(str)

In [11]:
off_train.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,nan,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [13]:
off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',keep_default_na=False)
off_train.columns=['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',keep_default_na=False)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
on_train = pd.read_csv('data/ccf_online_stage1_train.csv',keep_default_na=False)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']
off_train[['user_id','merchant_id','coupon_id']]=off_train[['user_id','merchant_id','coupon_id']].astype(str)
off_test[['user_id','merchant_id','coupon_id']]=off_test[['user_id','merchant_id','coupon_id']].astype(str)
on_train[['user_id','merchant_id','coupon_id']]=on_train[['user_id','merchant_id','coupon_id']].astype(str)

In [14]:
off_train.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [15]:
print('offline train date_received')
print(off_train[off_train['date_received']!='null']['date_received'].min())
print(off_train[off_train['date_received']!='null']['date_received'].max())

print('online train date_received')
print(on_train[on_train['date_received']!='null']['date_received'].min())
print(on_train[on_train['date_received']!='null']['date_received'].max())

print('test date_received')
print(off_test['date_received'].min())
print(off_test['date_received'].max())

offline train date_received
20160101
20160615
online train date_received
20160101
20160615
test date_received
20160701
20160731


In [16]:
#查看online offline 训练集的 user_id与测试集的重合度
off_train_user=off_train[['user_id']].copy().drop_duplicates()
off_test_user=off_test[['user_id']].copy().drop_duplicates()
on_train_user=on_train[['user_id']].copy().drop_duplicates()
print ('offline 训练集用户ID数量')
print (off_train_user.user_id.count())
print ('online 训练集用户ID数量')
print (on_train_user.user_id.count())
print ('offline 测试集用户ID数量')
print (off_test_user.user_id.count())
off_train_user['off_train_flag']=1
off_merge=off_test_user.merge(off_train_user,on='user_id', how="left").reset_index().fillna(0)
print ('offline 训练集用户与测试集用户重复数量')
print(off_merge['off_train_flag'].sum())
print ('offline 训练集用户与测试集重复用户在总测试集用户中的占比')
print(off_merge['off_train_flag'].sum()/off_merge['off_train_flag'].count())
on_train_user['on_train_flag']=1
on_merge=off_test_user.merge(on_train_user,on='user_id', how="left").reset_index().fillna(0)
print ('online 训练集用户与测试集用户重复数量')
print(on_merge['on_train_flag'].sum())
print ('online 训练集用户与测试集重复用户在总测试集用户中的占比')
print(on_merge['on_train_flag'].sum()/on_merge['on_train_flag'].count())

offline 训练集用户ID数量
539438
online 训练集用户ID数量
762858
offline 测试集用户ID数量
76309
offline 训练集用户与测试集用户重复数量
76307.0
offline 训练集用户与测试集重复用户在总测试集用户中的占比
0.9999737907717308
online 训练集用户与测试集用户重复数量
43155.0
online 训练集用户与测试集重复用户在总测试集用户中的占比
0.5655296229802513


In [17]:
#查看online offline 训练集的 merchant_id 与测试集的重合度
off_train_merchant=off_train[['merchant_id']].copy().drop_duplicates()
off_test_merchant=off_test[['merchant_id']].copy().drop_duplicates()
on_train_merchant=on_train[['merchant_id']].copy().drop_duplicates()
print ('offline 训练集商家ID数量')
print (off_train_merchant.merchant_id.count())
print ('online 训练集商家ID数量')
print (on_train_merchant.merchant_id.count())
print ('offline 测试集商家ID数量')
print (off_test_merchant.merchant_id.count())

off_train_merchant['off_train_flag']=1
off_merge=off_test_merchant.merge(off_train_merchant,on='merchant_id', how="left").reset_index().fillna(0)
print ('offline 训练集商家与测试集商家重复数量')
print(off_merge['off_train_flag'].sum())
print ('offline 训练集商家与测试集重复商家在总测试集商家中的占比')
print(off_merge['off_train_flag'].sum()/off_merge['off_train_flag'].count())

on_train_merchant['on_train_flag']=1
on_merge=off_test_merchant.merge(on_train_merchant,on='merchant_id', how="left").reset_index().fillna(0)
print ('online 训练集商家与测试集商家重复数量')
print(on_merge['on_train_flag'].sum())
print ('online 训练集商家与测试集重复商家在总测试集商家中的占比')
print(on_merge['on_train_flag'].sum()/on_merge['on_train_flag'].count())

offline 训练集商家ID数量
8415
online 训练集商家ID数量
7999
offline 测试集商家ID数量
1559
offline 训练集商家与测试集商家重复数量
1558.0
offline 训练集商家与测试集重复商家在总测试集商家中的占比
0.9993585631815266
online 训练集商家与测试集商家重复数量
0.0
online 训练集商家与测试集重复商家在总测试集商家中的占比
0.0


In [18]:
#查看online offline 训练集的 coupon_id 与测试集的重合度
off_train_coupon=off_train[['coupon_id']].copy().drop_duplicates()
off_test_coupon=off_test[['coupon_id']].copy().drop_duplicates()
on_train_coupon=on_train[['coupon_id']].copy().drop_duplicates()
print ('offline 训练集优惠券ID数量')
print (off_train_coupon.coupon_id.count())
print ('online 训练集优惠券ID数量')
print (on_train_coupon.coupon_id.count())
print ('offline 测试集优惠券ID数量')
print (off_test_coupon.coupon_id.count())

off_train_coupon['off_train_flag']=1
off_merge=off_test_coupon.merge(off_train_coupon,on='coupon_id', how="left").reset_index().fillna(0)
print ('offline 训练集优惠券与测试集优惠券重复数量')
print(off_merge['off_train_flag'].sum())
print ('offline 训练集优惠券与测试集重复优惠券在总测试集优惠券中的占比')
print(off_merge['off_train_flag'].sum()/off_merge['off_train_flag'].count())

on_train_coupon['on_train_flag']=1
on_merge=off_test_coupon.merge(on_train_coupon,on='coupon_id', how="left").reset_index().fillna(0)
print ('online 训练集优惠券与测试集优惠券重复数量')
print(on_merge['on_train_flag'].sum())
print ('online 训练集优惠券与测试集重复优惠券在总测试集优惠券中的占比')
print(on_merge['on_train_flag'].sum()/on_merge['on_train_flag'].count())

offline 训练集优惠券ID数量
9739
online 训练集优惠券ID数量
27748
offline 测试集优惠券ID数量
2050
offline 训练集优惠券与测试集优惠券重复数量
1118.0
offline 训练集优惠券与测试集重复优惠券在总测试集优惠券中的占比
0.5453658536585366
online 训练集优惠券与测试集优惠券重复数量
0.0
online 训练集优惠券与测试集重复优惠券在总测试集优惠券中的占比
0.0


对用户，商家，优惠券在训练集和测试集的重合情况进行探索发现： 测试集的用户ID与Offline训练集重复占比0.999以上，与Online训练集重复占比0.565。 测试集的商家ID与Offline训练集重复占比0.999以上，与Online训练集没有重复。 测试集的优惠券ID与训练集都没有重复。 
结论：Online数据价值比较低，后续特征提取将以Offline训练集为主。在提取优惠券统计特征的时候不能通过ID进行合并。 在后续可视化分析中将主要在Offline训练集及测试集之间进行。